# CONSULTA BD

In [ ]:
import pandas as pd
import os

# Detectar entorno para Google Colab
try:
    from google.colab import drive
    drive.mount('/content/drive')
except:
    pass

# Definir la ruta base para los archivos en Google Drive
RUTA_BASE = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/'

# Cargar el DataFrame de hermanos desde la ruta correcta
df_hermanos = pd.read_csv(os.path.join(RUTA_BASE, "base_datos_hermanos.csv"))

df_hermanos[(df_hermanos['genero'] == 'M') & (df_hermanos['privilegio'] == 'ANC')]



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,id,nombre_completo,genero,privilegio,activo,f_presidente,f_oracion,f_lectura,f_tesoros,f_perlas,f_vida,f_estudio_libro,f_lector_libro,f_discurso_est,f_demos,f_ayudante,observaciones
4,102,Juan Pablo Alfonso,M,ANC,1,1,1,0,1,1,1,1,1,0,1,1,NaN
13,11,James Cabieles,M,ANC,1,1,1,0,1,1,1,1,1,0,1,1,NaN
19,13,Cristian Calderón,M,ANC,1,1,1,0,1,1,1,1,1,0,1,1,NaN
29,58,William Diaz,M,ANC,1,1,1,0,1,1,1,1,1,1,1,1,NaN
47,3,Cesar Hernandez,M,ANC,1,1,1,0,1,1,1,1,1,0,1,1,NaN
49,1,Orlando Hernandez,M,ANC,1,1,1,0,1,1,1,1,1,0,1,1,NaN
68,9,Ricardo Morales,M,ANC,1,1,1,0,1,1,1,1,1,0,0,0,NaN
82,23,Edwin Quintero,M,ANC,1,1,1,0,1,1,1,1,1,1,1,1,NaN
86,27,Gustavo Ramirez,M,ANC,1,1,1,0,1,1,1,1,0,0,1,1,NaN
88,17,Juan Carlos Riaño,M,ANC,1,1,1,0,1,1,1,1,1,1,1,1,NaN


# 📥 FASE 1 v5.0: EXTRACTOR WEB ROBUSTO (WOL)

In [6]:
# =============================================================================
#  🕷️ FASE 1 v5.0: EXTRACTOR UNIVERSAL (CAZADOR AUTOMÁTICO)
# =============================================================================
#  MEJORAS:
#  1. Pide AÑO y MES al usuario (ya no hay que editar fechas a mano).
#  2. Calcula automáticamente los lunes de ese mes.
#  3. Mantiene la lógica "Cazador" para encontrar todas las partes (Maestros).
# =============================================================================

import requests
from bs4 import BeautifulSoup
import json
import re
import os
import calendar
from datetime import date

# Detectar entorno para Google Colab
try:
    from google.colab import drive
    drive.mount('/content/drive')
except:
    pass

# RUTA BASE (Verifica que sea correcta)
RUTA_BASE = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/'
FILE_JSON = os.path.join(RUTA_BASE, 'data_reuniones_2026.json')

def limpiar_texto(t): return t.replace('\u00a0', ' ').strip() if t else ""

def obtener_html(url):
    try:
        r = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, timeout=20)
        return BeautifulSoup(r.content, 'html.parser') if r.status_code == 200 else None
    except: return None

def generar_urls_lunes(anio, mes):
    """Genera las URLs de WOL para todos los lunes del mes dado"""
    urls = []
    cal = calendar.Calendar(firstweekday=calendar.MONDAY)
    for semana in cal.monthdatescalendar(anio, mes):
        lunes = semana[0]
        # Solo si el lunes pertenece al mes solicitado
        if lunes.month == mes:
            urls.append(f"https://wol.jw.org/es/wol/dt/r4/lp-s/{lunes.year}/{lunes.month}/{lunes.day}")
    return urls

def extractor_universal():
    print("🌍 GENERADOR DE PROGRAMA JW (Fase 1 v5.0)")
    print("=========================================")

    # INTERACCIÓN CON EL USUARIO
    try:
        anio_input = int(input("👉 Ingresa el AÑO (ej. 2026): "))
        mes_input = int(input("👉 Ingresa el MES (1=Ene, 2=Feb...): "))
    except ValueError:
        return print("❌ Error: Debes ingresar números válidos.")

    urls_a_procesar = generar_urls_lunes(anio_input, mes_input)
    print(f"\n   📅 Se detectaron {len(urls_a_procesar)} semanas para procesar.")

    datos = []

    for url in urls_a_procesar:
        print(f"\n   📥 Descargando: {url} ...")
        soup = obtener_html(url)
        if not soup:
            print("      ❌ Error de conexión.")
            continue

        # 1. INFO GENERAL
        elem_titulo = soup.find('h1', id='p1')
        txt_sem = limpiar_texto(elem_titulo.get_text()) if elem_titulo else "Semana desconocida"
        print(f"      🗓️ {txt_sem}")

        # CANCIONES
        canciones = []
        for c in soup.find_all(['p', 'div']): # A veces cambia el tag
             txt = c.get_text()
             if "Canción" in txt or "Cántico" in txt:
                 num = re.search(r'(\d+)', txt)
                 if num: canciones.append(num.group(1))

        # Lógica simple para asignar (Inicio, Medio, Final)
        c_ini = canciones[0] if len(canciones) > 0 else "---"
        c_med = canciones[1] if len(canciones) > 1 else "---"
        c_fin = canciones[2] if len(canciones) > 2 else "---"

        partes = []
        partes.append({"tipo": "Presidente", "titulo": "Presidente de la Reunión"})
        partes.append({"tipo": "Oracion_Inicio", "titulo": f"Canción {c_ini} y oración"})

        # 2. TESOROS
        tesoros_encontrado = False
        for elem in soup.find_all(['p', 'h3']):
            txt = limpiar_texto(elem.get_text())
            if "(10 mins.)" in txt and "Perlas" not in txt and "Consejo" not in txt:
                tit = txt.replace("(10 mins.)", "").strip()
                partes.append({"tipo": "Tesoros", "titulo": tit})
                tesoros_encontrado = True
                break

        if not tesoros_encontrado: partes.append({"tipo": "Tesoros", "titulo": "Discurso de Tesoros"})
        partes.append({"tipo": "Perlas", "titulo": "Busquemos Perlas Escondidas"})

        # LECTURA
        lectura_tit = "Lectura de la Biblia"
        for elem in soup.find_all(['p', 'h3']):
            if "(4 mins.)" in elem.get_text():
                match = re.search(r'\)\s*(.+)', elem.get_text())
                if match: lectura_tit = match.group(1).strip()
                break
        partes.append({"tipo": "Lectura", "titulo": lectura_tit})

        # 3. MAESTROS (Modo Cazador)
        claves = ["Empiece", "Haga revisitas", "Haga discípulos", "Explique", "Discurso"]
        h3s = soup.find_all('h3')
        count_m = 0
        for h3 in h3s:
            txt = limpiar_texto(h3.get_text())
            if "MAESTROS" in txt.upper() or "Estudio" in txt: continue

            if any(k in txt for k in claves):
                # Limpiar texto
                tit_clean = re.split(r'\(\d', txt)[0].strip().strip('.:0123456789')
                partes.append({"tipo": "Maestros", "titulo": tit_clean})
                count_m += 1
                print(f"         ✅ Maestro: {tit_clean}")

        if count_m == 0:
            print("         ⚠️ ALERTA: No se encontraron partes de Maestros. Verifica el HTML.")

        # 4. VIDA CRISTIANA (NVC)
        secc_vida = soup.find('div', id='section4')
        if secc_vida:
            for h3 in secc_vida.find_all('h3'):
                txt = limpiar_texto(h3.get_text())
                if any(x in txt for x in ["VIDA", "Cántico", "Oración", "Estudio", "Conclusión"]): continue
                tit = re.split(r'\(\d', txt)[0].strip()
                partes.append({"tipo": "NVC", "titulo": tit})

        # FIJOS FINALES
        titulo_ebc = "Estudio Bíblico de Congregación"
        # Intento extra de capturar título del libro
        for h3 in soup.find_all('h3'):
            if "Estudio bíblico" in h3.get_text():
                match = re.search(r'\)\s*(.+)', h3.get_text())
                if match: titulo_ebc = match.group(1).strip()

        partes.append({"tipo": "Estudio_Libro", "titulo": titulo_ebc})
        partes.append({"tipo": "Lector_Libro", "titulo": "Lectura del Estudio Bíblico"})
        partes.append({"tipo": "Oracion_Final", "titulo": f"Canción {c_fin} y oración"})

        datos.append({
            "semana": txt_sem,
            "cancion_inicio": c_ini,
            "cancion_medio": c_med,
            "cancion_final": c_fin,
            "partes": partes
        })

    # GUARDAR
    # Modo 'w' sobrescribe el archivo, perfecto para limpiar errores previos
    with open(FILE_JSON, 'w', encoding='utf-8') as f:
        json.dump(datos, f, indent=4, ensure_ascii=False)

    print(f"\n💾 ¡LISTO! JSON generado para {mes_input}/{anio_input}")
    print(f"   Archivo: {os.path.basename(FILE_JSON)}")

if __name__ == "__main__":
    extractor_universal()

🌍 GENERADOR DE PROGRAMA JW (Fase 1 v5.0)
👉 Ingresa el AÑO (ej. 2026): 
❌ Error: Debes ingresar números válidos.


# 🤖 FASE 2 v2.3: GENERADOR AUTOMÁTICO DE ASGINACIONES:

In [1]:
# =============================================================================
#  🤖 FASE 2 v3.0: GENERADOR DE ASIGNACIONES (CORREGIDO Y BLINDADO)
# =============================================================================
import json
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

# Detectar entorno para Google Colab
try:
    from google.colab import drive
    drive.mount('/content/drive')
except:
    pass

# --- RUTAS ---
RUTA_BASE = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/'
FILES = {
    "DB_HERMANOS": os.path.join(RUTA_BASE, "base_datos_hermanos.csv"),
    "JSON_REUNIONES": os.path.join(RUTA_BASE, "data_reuniones_2026.json"),
    "HISTORIAL": os.path.join(RUTA_BASE, "historial_asignaciones.csv"),
    "OUTPUT_JSON": os.path.join(RUTA_BASE, "programa_completo_asignado.json"),
    "AUSENCIAS": os.path.join(RUTA_BASE, "ausencias.csv")
}

FECHA_INICIO_PROGRAMA = datetime(2026, 1, 5)

# MAPEO DE HABILIDADES
MAPEO_HABILIDADES = {
    'Presidente': 'f_presidente', 'Oracion': 'f_oracion', 'Tesoros': 'f_tesoros',
    'Perlas': 'f_perlas', 'Lectura': 'f_lectura', 'Maestros': 'f_demos',
    'Vida': 'f_vida', 'Estudio_Libro': 'f_estudio_libro', 'Lector_Libro': 'f_lector_libro',
    'Ayudante': 'f_ayudante'
}
SOLO_VARONES = ['Presidente', 'Oracion', 'Tesoros', 'Lectura', 'Estudio_Libro', 'Lector_Libro']

class MotorHibridoV3:
    def __init__(self):
        self.db_hermanos = pd.DataFrame()
        self.historial = pd.DataFrame()
        self.ausencias = pd.DataFrame()
        self.data_reuniones = []
        self.cargar_datos()

    def cargar_datos(self):
        print("📥 INICIALIZANDO GENERADOR V3.0 (Corrección de Bugs)...")

        # 1. DB Hermanos (Forzamos ID a entero)
        if os.path.exists(FILES["DB_HERMANOS"]):
            self.db_hermanos = pd.read_csv(FILES["DB_HERMANOS"])
            self.db_hermanos['id'] = pd.to_numeric(self.db_hermanos['id'], errors='coerce').fillna(0).astype(int)
            for col in [c for c in self.db_hermanos.columns if c.startswith('f_')]:
                self.db_hermanos[col] = pd.to_numeric(self.db_hermanos[col], errors='coerce').fillna(0)

        # 2. Historial (Limpieza de NaN y Fechas)
        if os.path.exists(FILES["HISTORIAL"]):
            self.historial = pd.read_csv(FILES["HISTORIAL"])
            self.historial['fecha_asignacion'] = pd.to_datetime(self.historial['fecha_asignacion'])
            # Eliminar filas vacías críticas
            self.historial = self.historial.dropna(subset=['nombre_hermano'])
        else:
            self.historial = pd.DataFrame(columns=['fecha_asignacion', 'id_hermano', 'tipo_asignacion'])

        # 3. Ausencias (Conversión estricta)
        if os.path.exists(FILES["AUSENCIAS"]):
            self.ausencias = pd.read_csv(FILES["AUSENCIAS"])
            self.ausencias['fecha_inicio'] = pd.to_datetime(self.ausencias['fecha_inicio'])
            self.ausencias['fecha_fin'] = pd.to_datetime(self.ausencias['fecha_fin'])
            self.ausencias['id_hermano'] = pd.to_numeric(self.ausencias['id_hermano'], errors='coerce').fillna(0).astype(int)
        else:
            self.ausencias = pd.DataFrame(columns=['id_hermano', 'fecha_inicio', 'fecha_fin'])

        # 4. JSON
        if os.path.exists(FILES["JSON_REUNIONES"]):
            with open(FILES["JSON_REUNIONES"], 'r', encoding='utf-8') as f:
                self.data_reuniones = json.load(f)

    def esta_ausente(self, id_hermano, fecha_reunion):
        """Verifica si el hermano está ausente en esa fecha específica."""
        if self.ausencias.empty: return False

        # Filtro vectorizado
        filtro = (self.ausencias['id_hermano'] == id_hermano) & \
                 (self.ausencias['fecha_inicio'] <= fecha_reunion) & \
                 (self.ausencias['fecha_fin'] >= fecha_reunion)

        return not self.ausencias[filtro].empty

    def verificar_manual_consumible(self, fecha_dt, tipo_parte, indices_usados):
        """
        Busca en el historial pero IGNORA las filas ya usadas (indices_usados).
        Retorna: nombre, id, indice_del_historial
        """
        if self.historial.empty: return None, None, None

        # Filtramos por fecha y tipo
        filtro = (self.historial['fecha_asignacion'] == fecha_dt) & \
                 (self.historial['tipo_asignacion'] == tipo_parte)

        matches = self.historial[filtro]

        # Iteramos sobre los matches para encontrar uno NO usado
        for idx, row in matches.iterrows():
            if idx not in indices_usados:
                nombre = row['nombre_hermano']
                # Validación extra para evitar NaN
                if pd.isna(nombre) or str(nombre).strip() == "" or str(nombre).lower() == "nan":
                    continue
                return nombre, row['id_hermano'], idx

        return None, None, None

    def obtener_flag(self, tipo, titulo):
        if 'Maestros' in tipo: return 'f_ayudante' if 'Ayudante' in titulo else 'f_demos'
        for k, v in MAPEO_HABILIDADES.items():
            if k in tipo: return v
        return None

    def buscar_candidato_auto(self, tipo_parte, titulo_parte, fecha_dt, ids_excluidos):
        candidatos = self.db_hermanos[self.db_hermanos['activo'] == 1].copy()

        # 1. Filtro Habilidad
        flag = self.obtener_flag(tipo_parte, titulo_parte)
        if flag and flag in candidatos.columns: candidatos = candidatos[candidatos[flag] == 1]

        # 2. Filtro Género
        es_varon = any(x in tipo_parte for x in SOLO_VARONES)
        if 'Maestros' in tipo_parte and 'Discurso' in titulo_parte: es_varon = True
        if es_varon: candidatos = candidatos[candidatos['genero'] == 'M']

        # 3. Filtro Ausencia (Corrección Bug 3)
        candidatos['ausente'] = candidatos['id'].apply(lambda x: self.esta_ausente(x, fecha_dt))
        candidatos = candidatos[candidatos['ausente'] == False]

        # 4. Exclusión de usados hoy
        candidatos = candidatos[~candidatos['id'].isin(ids_excluidos)]

        if candidatos.empty: return None, None

        # 5. Ranking LRU
        ranking = []
        for _, h in candidatos.iterrows():
            hist_h = self.historial[self.historial['id_hermano'] == h['id']]
            if hist_h.empty: ultima = datetime(2020, 1, 1)
            else: ultima = hist_h['fecha_asignacion'].max()
            ranking.append({'id': h['id'], 'nombre': h['nombre_completo'], 'ultima': ultima})

        df_rank = pd.DataFrame(ranking).sort_values(by='ultima', ascending=True)
        if not df_rank.empty:
            # Randomizado ligero entre los 3 mejores para variedad
            seleccionado = df_rank.head(3).sample(1).iloc[0]
            return seleccionado['id'], seleccionado['nombre']

        return None, None

    def procesar(self):
        print("⚙️ EJECUTANDO GENERADOR V3.0...")
        programa_final = []

        for i, semana in enumerate(self.data_reuniones):
            fecha_dt = FECHA_INICIO_PROGRAMA + timedelta(weeks=i)
            print(f"\n📅 Procesando: {semana['semana']} ({fecha_dt.date()})")

            partes_asignadas = []
            ids_usados_hoy = []
            indices_historial_usados = [] # Para evitar los "Trillizos"

            # Pre-cargar manuales de hoy en ids_usados (para no repetir el mismo hermano en otra parte auto)
            manuales_hoy = self.historial[self.historial['fecha_asignacion'] == fecha_dt]
            ids_usados_hoy = manuales_hoy['id_hermano'].dropna().astype(int).tolist()

            for parte in semana.get('partes', []):
                tipo = parte['tipo']
                titulo = parte.get('titulo', tipo)
                parte_new = parte.copy()

                # --- 1. ASIGNACIÓN DEL ESTUDIANTE / PRINCIPAL ---

                # A. Check Historial (Consumible)
                nom_m, id_m, idx_h = self.verificar_manual_consumible(fecha_dt, tipo, indices_historial_usados)

                if nom_m:
                    parte_new['asignado_a'] = nom_m
                    parte_new['origen'] = 'MANUAL_HIST'
                    indices_historial_usados.append(idx_h) # ¡Marcamos como usado! (Fix Bug 1)
                    if id_m and id_m not in ids_usados_hoy: ids_usados_hoy.append(id_m)
                    print(f"   ✅ Historial: {tipo} -> {nom_m}")
                else:
                    # B. Auto Fill (Si no hay manual)
                    id_a, nom_a = self.buscar_candidato_auto(tipo, titulo, fecha_dt, ids_usados_hoy)
                    if nom_a:
                        parte_new['asignado_a'] = nom_a
                        parte_new['origen'] = 'AUTO'
                        ids_usados_hoy.append(id_a)

                        # Registrar en memoria temporal para rotación
                        new_row = {'fecha_asignacion': fecha_dt, 'id_hermano': id_a, 'tipo_asignacion': tipo, 'nombre_hermano': nom_a}
                        self.historial = pd.concat([self.historial, pd.DataFrame([new_row])], ignore_index=True)
                        print(f"   🤖 Auto: {tipo} -> {nom_a}")
                    else:
                        parte_new['asignado_a'] = "PENDIENTE"
                        parte_new['origen'] = "VACIO"

                # --- 2. ASIGNACIÓN DEL AYUDANTE (Solo para Maestros NO Discurso) ---
                if 'Maestros' in tipo and 'Discurso' not in titulo:
                    # Intentar buscar "Ayudante" en el historial
                    nom_ayu, id_ayu, idx_ayu = self.verificar_manual_consumible(fecha_dt, 'Ayudante', indices_historial_usados)

                    if nom_ayu:
                        parte_new['ayudante'] = nom_ayu # Inyectamos campo ayudante
                        indices_historial_usados.append(idx_ayu)
                        print(f"      └── 🤝 Ayudante (Historial): {nom_ayu}")

                    # (Opcional: Si quieres que asigne ayudante automático también,
                    # aquí iría la llamada a buscar_candidato_auto con flag 'f_ayudante')

                partes_asignadas.append(parte_new)

            semana['partes'] = partes_asignadas
            programa_final.append(semana)

        # Guardar JSON Final
        with open(FILES["OUTPUT_JSON"], 'w', encoding='utf-8') as f:
            json.dump(programa_final, f, indent=4, ensure_ascii=False)
        print(f"\n💾 Borrador generado: {os.path.basename(FILES['OUTPUT_JSON'])}")

if __name__ == "__main__":
    app = MotorHibridoV3()
    app.procesar()

Mounted at /content/drive
📥 INICIALIZANDO GENERADOR V3.0 (Corrección de Bugs)...
⚙️ EJECUTANDO GENERADOR V3.0...

📅 Procesando: 5-11 DE ENERO (2026-01-05)
   🤖 Auto: Presidente -> Juan Pablo Alfonso
   🤖 Auto: Oracion_Inicio -> Andrés Cárdenas
   🤖 Auto: Tesoros -> James Cabieles
   🤖 Auto: Perlas -> Manuel Céspedes
   ✅ Historial: Lectura -> Duvan Torres
   ✅ Historial: Maestros -> Dora Mendoza
      └── 🤝 Ayudante (Historial): Clara Parra
   ✅ Historial: Maestros -> María Montañez
      └── 🤝 Ayudante (Historial): Marisol Vargas
   ✅ Historial: Maestros -> Jimmy Pacheco
   🤖 Auto: Estudio_Libro -> Ricardo Morales
   🤖 Auto: Lector_Libro -> John Herrera
   🤖 Auto: Oracion_Final -> Vicente Castro

📅 Procesando: 12-18 DE ENERO (2026-01-12)
   🤖 Auto: Presidente -> Gustavo Ramirez
   🤖 Auto: Oracion_Inicio -> Jorge Alba
   🤖 Auto: Tesoros -> Cesar Hernandez
   🤖 Auto: Perlas -> Paul Garcia
   🤖 Auto: Lectura -> Juan Camilo Mendoza
   🤖 Auto: Maestros -> Elvia Silva
   ✅ Historial: Maestr

# ✍️ FASE 3 v4.0: EDITOR MAESTRO

In [1]:
# =============================================================================
#  ✏️ FASE 3 (FINAL): EDITOR MAESTRO VMC (Con Ausencias + Edición JSON)
# =============================================================================
import pandas as pd
import json
import os
from datetime import datetime, timedelta

# --- RUTAS ---
RUTA_BASE = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/'
FILES = {
    "DB": os.path.join(RUTA_BASE, 'base_datos_hermanos.csv'),
    "HIST": os.path.join(RUTA_BASE, 'historial_asignaciones.csv'),
    "JSON_OUT": os.path.join(RUTA_BASE, 'programa_completo_asignado.json'), # Lee el YA asignado
    "AUSENCIAS": os.path.join(RUTA_BASE, 'ausencias.csv')
}
FECHA_BASE = datetime(2026, 1, 5) # Ajustar primer lunes del programa

# MAPEO HABILIDADES
MAPEO = {
    'Presidente': 'f_presidente', 'Oracion': 'f_oracion', 'Tesoros': 'f_tesoros',
    'Perlas': 'f_perlas', 'Lectura': 'f_lectura', 'Maestros': 'f_demos',
    'Vida': 'f_vida', 'Estudio_Libro': 'f_estudio_libro', 'Lector_Libro': 'f_lector_libro',
    'Ayudante': 'f_ayudante'
}
SOLO_VARONES = ['Presidente', 'Oracion', 'Tesoros', 'Lectura', 'Estudio_Libro', 'Lector_Libro']

class CerebroTeocratico:
    def __init__(self):
        self.db = pd.DataFrame()
        self.hist = pd.DataFrame()
        self.ausencias = pd.DataFrame()
        self.cargar_datos()

    def cargar_datos(self):
        # 1. Base Hermanos
        if os.path.exists(FILES["DB"]):
            self.db = pd.read_csv(FILES["DB"])
            for c in [x for x in self.db.columns if x.startswith('f_')]:
                self.db[c] = pd.to_numeric(self.db[c], errors='coerce').fillna(0)

        # 2. Historial
        if os.path.exists(FILES["HIST"]):
            self.hist = pd.read_csv(FILES["HIST"])
            self.hist['fecha_asignacion'] = pd.to_datetime(self.hist['fecha_asignacion'])
        else:
            self.hist = pd.DataFrame(columns=['fecha_asignacion', 'id_hermano', 'nombre_hermano', 'tipo_asignacion'])

        # 3. Ausencias (NUEVO)
        if os.path.exists(FILES["AUSENCIAS"]):
            self.ausencias = pd.read_csv(FILES["AUSENCIAS"])
            # Convertir fechas
            self.ausencias['fecha_inicio'] = pd.to_datetime(self.ausencias['fecha_inicio'], errors='coerce')
            self.ausencias['fecha_fin'] = pd.to_datetime(self.ausencias['fecha_fin'], errors='coerce')
        else:
            self.ausencias = pd.DataFrame(columns=['id_hermano', 'fecha_inicio', 'fecha_fin'])

    def esta_ausente(self, id_hermano, fecha_reunion):
        """Verifica si la fecha cae en rango de ausencia."""
        if self.ausencias.empty: return False

        filtro = (self.ausencias['id_hermano'] == id_hermano) & \
                 (self.ausencias['fecha_inicio'] <= fecha_reunion) & \
                 (self.ausencias['fecha_fin'] >= fecha_reunion)

        return not self.ausencias[filtro].empty

    def obtener_sugerencias(self, tipo, titulo, fecha_dt, ids_excluidos=[]):
        candidatos = self.db[self.db['activo'] == 1].copy()

        # A. Filtro Habilidad
        flag = 'f_demos'
        if 'Ayudante' in tipo: flag = 'f_ayudante'
        else:
            for k, v in MAPEO.items():
                if k in tipo: flag = v; break

        if flag in candidatos.columns: candidatos = candidatos[candidatos[flag] == 1]

        # B. Filtro Género
        es_varon = any(x in tipo for x in SOLO_VARONES)
        if 'Maestros' in tipo and 'Discurso' in titulo: es_varon = True
        if es_varon: candidatos = candidatos[candidatos['genero'] == 'M']

        # C. Filtro Ausencia (NUEVO)
        candidatos['ausente'] = candidatos['id'].apply(lambda x: self.esta_ausente(x, fecha_dt))
        candidatos = candidatos[candidatos['ausente'] == False]

        # D. Exclusión y Ranking
        candidatos = candidatos[~candidatos['id'].isin(ids_excluidos)]

        ranking = []
        for _, h in candidatos.iterrows():
            hist_h = self.hist[self.hist['id_hermano'] == h['id']]
            if hist_h.empty:
                dias = 999
            else:
                ultima = hist_h['fecha_asignacion'].max()
                dias = (datetime.now() - ultima).days
            ranking.append({'id': h['id'], 'nombre': h['nombre_completo'], 'dias': dias})

        return pd.DataFrame(ranking).sort_values(by='dias', ascending=False).head(5)

# --- INTERFAZ DE EDICIÓN ---
def editor_maestro():
    print("📝 EDITOR MAESTRO VMC (Edita JSON + Guarda Historial)")

    cerebro = CerebroTeocratico()

    # Cargar JSON generado por Fase 2 (Auto)
    if not os.path.exists(FILES["JSON_OUT"]):
        return print("❌ No existe 'programa_completo_asignado.json'. Corre primero el Generador (Fase 2).")

    with open(FILES["JSON_OUT"], 'r', encoding='utf-8') as f:
        programa = json.load(f)

    # SELECCIÓN DE SEMANA
    print("\n📅 SEMANAS DISPONIBLES:")
    for i, s in enumerate(programa):
        print(f"   {i+1}. {s['semana']}")

    try:
        idx = int(input("\n👉 Elige Semana: ")) - 1
        semana = programa[idx]
    except: return

    fecha_sem = FECHA_BASE + timedelta(weeks=idx)
    print(f"\n🔧 Editando: {semana['semana']} (Fecha ref: {fecha_sem.date()})")

    # EDICIÓN
    cambios_json = False
    nuevos_historial = []

    for parte in semana['partes']:
        tipo = parte['tipo']
        titulo = parte.get('titulo', tipo)
        asignado = parte.get('asignado_a', 'VACÍO')
        origen = parte.get('origen', 'MANUAL')

        print(f"\n🔹 {titulo} ({tipo})")
        print(f"   👤 ASIGNADO ACTUAL: {asignado} [{origen}]")

        accion = input("   👉 ENTER mantener | '?' sugerir | Nombre buscar: ").strip()

        if not accion: continue # Mantener actual

        id_sel, nom_sel = None, None

        # MODO SUGERENCIA
        if accion == '?':
            sugs = cerebro.obtener_sugerencias(tipo, titulo, fecha_sem)
            if sugs.empty:
                print("      ⚠️ Nadie disponible (Revisar ausencias/requisitos).")
            else:
                print(f"      {'ID':<3} {'NOMBRE':<25} {'DESCANSÓ'}")
                lista = sugs.values.tolist()
                for i, h in enumerate(lista): print(f"      [{i}] {h[1]:<25} {h[2]} días")

                sel = input("      Selecciona #: ")
                if sel.isdigit() and int(sel) < len(lista):
                    id_sel, nom_sel = lista[int(sel)][0], lista[int(sel)][1]

        # MODO BÚSQUEDA
        else:
            busq = cerebro.db[cerebro.db['nombre_completo'].str.contains(accion, case=False)]
            if not busq.empty:
                lista = busq.values.tolist()
                for i, h in enumerate(lista):
                    # Ajustar índices según tu CSV: id=0, nombre=1 (revisar tu estructura real)
                    # Basado en tu snippet: id, nombre_completo, genero...
                    print(f"      [{i}] {h[1]}") # h[1] es nombre_completo
                sel = input("      Selecciona #: ")
                if sel.isdigit():
                    # Como es dataframe, usamos iloc sobre el filtrado
                    fila = busq.iloc[int(sel)]
                    id_sel, nom_sel = fila['id'], fila['nombre_completo']

        # APLICAR CAMBIO
        if id_sel:
            # 1. Actualizar JSON en memoria
            parte['asignado_a'] = nom_sel
            parte['origen'] = 'MANUAL_EDITOR'
            cambios_json = True

            # 2. Preparar Historial
            nuevos_historial.append({
                'fecha_asignacion': fecha_sem.strftime('%Y-%m-%d'),
                'id_hermano': id_sel,
                'nombre_hermano': nom_sel,
                'tipo_asignacion': tipo
            })
            print(f"   ✅ Cambiado a: {nom_sel}")

            # Lógica Ayudante (Simplificada)
            if 'Maestros' in tipo and 'Discurso' not in titulo:
                # Preguntar ayudante si cambió el estudiante
                print("      ¿Cambiar Ayudante también?")
                # ... (Lógica similar para ayudante si se desea)

    # GUARDADO FINAL
    if cambios_json:
        print("\n💾 GUARDANDO CAMBIOS...")

        # A. Guardar JSON
        with open(FILES["JSON_OUT"], 'w', encoding='utf-8') as f:
            json.dump(programa, f, indent=4, ensure_ascii=False)
        print("   ✅ Archivo JSON actualizado (Listo para PDF).")

        # B. Guardar Historial
        if nuevos_historial:
            df_new = pd.DataFrame(nuevos_historial)
            # Agregar cabecera solo si no existe
            hdr = not os.path.exists(FILES["HIST"])
            df_new.to_csv(FILES["HIST"], mode='a', header=hdr, index=False)
            print("   ✅ Historial actualizado (Memoria futura).")
    else:
        print("\n👋 Saliste sin hacer cambios.")

if __name__ == "__main__":
    editor_maestro()

📝 EDITOR MAESTRO VMC (Edita JSON + Guarda Historial)
❌ No existe 'programa_completo_asignado.json'. Corre primero el Generador (Fase 2).


# Utilidad 1: Crear archivo de Ausencias


In [ ]:
# =============================================================================
#  🛠️ UTILIDAD: INICIALIZAR AUSENCIAS (CORREGIDO)
# =============================================================================
#  Crea el archivo ausencias.csv manejando la conexión a Google Drive.
# =============================================================================

import pandas as pd
import os
from google.colab import drive # 1. Importamos la librería de conexión

def inicializar_ausencias():
    print("🚑 INICIALIZANDO SISTEMA DE AUSENCIAS")
    print("-------------------------------------")

    # 2. VERIFICACIÓN DE CONEXIÓN (El paso clave que faltaba)
    if not os.path.exists('/content/drive'):
        print("🔌 Conectando a Google Drive...")
        drive.mount('/content/drive')
    else:
        print("✅ Google Drive ya está conectado.")

    # 3. RUTAS
    # Verifica que esta ruta sea la correcta (sin espacios extraños)
    RUTA_BASE = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/'
    FILE_AUSENCIAS = os.path.join(RUTA_BASE, 'ausencias.csv')

    # Validar que la carpeta madre exista
    if not os.path.exists(RUTA_BASE):
        print(f"❌ ERROR: No encuentro la carpeta del proyecto en:")
        print(f"   {RUTA_BASE}")
        return

    # 4. CREACIÓN DEL ARCHIVO
    if not os.path.exists(FILE_AUSENCIAS):
        # Columnas estándar para gestionar vacaciones
        cols = ['id_hermano', 'nombre', 'fecha_inicio', 'fecha_fin', 'motivo']

        df = pd.DataFrame(columns=cols)
        df.to_csv(FILE_AUSENCIAS, index=False, encoding='utf-8')

        print(f"\n✅ ¡ÉXITO! Archivo creado: {os.path.basename(FILE_AUSENCIAS)}")
        print("   Ahora puedes registrar las fechas en que los hermanos no estarán.")
    else:
        print(f"\n⚠️ El archivo ya existe en: {os.path.basename(FILE_AUSENCIAS)}")
        print("   No se ha borrado nada.")

if __name__ == "__main__":
    inicializar_ausencias()

🚑 INICIALIZANDO SISTEMA DE AUSENCIAS
-------------------------------------
✅ Google Drive ya está conectado.

✅ ¡ÉXITO! Archivo creado: ausencias.csv
   Ahora puedes registrar las fechas en que los hermanos no estarán.


# Uilidad 2: Agregar hermano nuevo

In [ ]:
# =============================================================================
#  🛠️ GESTIÓN DE HERMANOS: AGREGAR NUEVO (SIN ROMPER IDs)
# =============================================================================

import pandas as pd
import os

RUTA_BD = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/base_datos_hermanos.csv'

def agregar_hermano_manual():
    print("👤 INSCRIPCIÓN DE NUEVO HERMANO")
    print("-------------------------------")

    # 1. Cargar base de datos actual
    if not os.path.exists(RUTA_BD):
        print("❌ Error: No existe la base de datos.")
        return

    df = pd.read_csv(RUTA_BD)

    # 2. Calcular ID Automático (Max ID + 1) para integridad
    # Si la tabla está vacía, empieza en 1. Si no, busca el máximo.
    nuevo_id = 1
    if not df.empty:
        nuevo_id = df['id'].max() + 1

    print(f"🆔 ID Asignado Automáticamente: {nuevo_id}")

    # 3. Pedir datos
    nombre = input("Nombre Completo: ").strip().title()
    genero = input("Género (M/F): ").strip().upper()
    privilegio = input("Privilegio (PUB/SM/ANC): ").strip().upper()

    # Capacidades básicas (Preguntas rápidas)
    print("\n--- Capacidades (S/N) ---")
    lectura = 1 if input("¿Hace Lectura de Biblia? ").upper() == 'S' else 0
    demos = 1 if input("¿Hace Demostraciones/Ayudante? ").upper() == 'S' else 0
    discurso = 1 if input("¿Hace Discursos Estudiantiles? ").upper() == 'S' else 0

    # 4. Crear el diccionario con la estructura EXACTA
    nuevo_hermano = {
        'id': nuevo_id,
        'nombre_completo': nombre,
        'genero': genero,
        'privilegio': privilegio,
        'activo': 1,
        # Banderas
        'f_presidente': 0, 'f_oracion': 0,
        'f_lectura': lectura,
        'f_tesoros': 0, 'f_perlas': 0, 'f_vida': 0,
        'f_estudio_libro': 0, 'f_lector_libro': 0,
        'f_discurso_est': discurso,
        'f_demos': demos,
        'f_ayudante': demos, # Generalmente si hace demos, es ayudante
        'observaciones': "Ingreso manual"
    }

    # 5. Guardar
    # Convertimos el diccionario a DataFrame y lo concatenamos
    nuevo_df = pd.DataFrame([nuevo_hermano])

    # Alineamos columnas (para evitar desorden si faltan claves)
    # Usamos las columnas del df original como molde
    for col in df.columns:
        if col not in nuevo_df.columns:
            nuevo_df[col] = 0 # Rellenar con 0 lo que no preguntamos

    df_final = pd.concat([df, nuevo_df], ignore_index=True)
    df_final.to_csv(RUTA_BD, index=False, encoding='utf-8-sig')

    print(f"\n✅ ¡Listo! {nombre} ha sido inscrito con el ID {nuevo_id}.")

# Para usarlo, simplemente ejecuta:
# agregar_hermano_manual()

# Utilidad 3: MIGRADOR DE HISTORIAL (LEGACY -> SISTEMA NUEVO)

In [ ]:
# =============================================================================
#  🕰️ MIGRADOR DE HISTORIAL v3.0 (RUTA FORZADA)
# =============================================================================

import pandas as pd
import os
import re
from datetime import datetime

# --- CONFIGURACIÓN ---
# Usamos la misma ruta que ya sabemos que funciona para el JSON
RUTA_BASE = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/'

# PEGA AQUÍ EL NOMBRE EXACTO QUE SALIÓ EN EL DIAGNÓSTICO
NOMBRE_HISTORICO = 'historico_excel_asginaciones.csv'
NOMBRE_DB = 'base_datos_hermanos.csv'
NOMBRE_SALIDA = 'historial_asignaciones.csv'

# --- MAPEO (Tu diccionario de traducción) ---
MAPEO_COLUMNAS = {
    'PRESIDENCIA': 'Presidente',
    'ORACIÓN': 'Oracion_Inicio',
    'TESOROS DE LA BIBLIA': 'Tesoros',
    'BUSQUEMOS PERLAS ESCONDIDAS': 'Perlas',
    'LECTURA DE LA BIBLIA': 'Lectura',
    'SMM ASIG 1 ESTUD': 'Maestros', 'SMM ASIG 1 ACOMP': 'Ayudante',
    'SMM ASIG 2 ESTUD': 'Maestros', 'SMM ASIG 2 ACOMP': 'Ayudante',
    'SMM ASIG 3 ESTUD': 'Maestros', 'SMM ASIG 3 ACOMP': 'Ayudante',
    'SMM ASIG 4 ESTUD': 'Maestros', 'SMM ASIG 4 ACOMP': 'Ayudante',
    'NVC PARTE 1': 'NVC', 'NVC PARTE 2': 'NVC',
    'ESTUDIO LIBRO': 'Estudio_Libro', 'LECTOR LIBRO': 'Lector_Libro',
    'ORACIÓN FINAL': 'Oracion_Final'
}

def buscar_id_hermano(nombre_excel, df_db):
    if pd.isna(nombre_excel) or str(nombre_excel).strip() in ['', 'nan']: return None, None
    nombre_limpio = str(nombre_excel).strip()

    # 1. Exacto
    res = df_db[df_db['nombre_completo'].str.lower() == nombre_limpio.lower()]
    if not res.empty: return res.iloc[0]['id'], res.iloc[0]['nombre_completo']

    # 2. Parcial
    res = df_db[df_db['nombre_completo'].str.contains(nombre_limpio, case=False, regex=False)]
    if not res.empty: return res.iloc[0]['id'], res.iloc[0]['nombre_completo']

    return None, None

def extraer_fecha_lunes(texto_semana, mes_texto):
    try:
        dia = int(re.search(r'(\d+)', str(texto_semana)).group(1))
        mes_map = {'SEPTIEMBRE': 9, 'OCTUBRE': 10, 'NOVIEMBRE': 11, 'DICIEMBRE': 12, 'ENERO': 1, 'FEBRERO': 2}
        mes_num = mes_map.get(str(mes_texto).upper().strip(), 1)
        anio = 2026 if mes_num < 9 else 2025
        return f"{anio}-{mes_num:02d}-{dia:02d}"
    except:
        return datetime.now().strftime("%Y-%m-%d")

def ejecutar_migracion():
    print("🚀 INICIANDO MIGRACIÓN v3.0")

    # Construir rutas completas
    path_old = os.path.join(RUTA_BASE, NOMBRE_HISTORICO)
    path_db = os.path.join(RUTA_BASE, NOMBRE_DB)
    path_out = os.path.join(RUTA_BASE, NOMBRE_SALIDA)

    # Verificación estricta
    if not os.path.exists(path_old):
        print(f"❌ ERROR: Python NO ve el archivo en: {path_old}")
        print("   Por favor ejecuta el script de Diagnóstico primero.")
        return

    print(f"   ✅ Archivo encontrado: {NOMBRE_HISTORICO}")

    df_old = pd.read_csv(path_old)
    df_db = pd.read_csv(path_db)

    nuevos = []

    for _, fila in df_old.iterrows():
        semana = fila.get('SEMANA', 'Semana X')
        fecha = extraer_fecha_lunes(semana, fila.get('MES', ''))

        for col_orig, tipo_dest in MAPEO_COLUMNAS.items():
            if col_orig in df_old.columns:
                nombre = fila[col_orig]
                id_h, nombre_real = buscar_id_hermano(nombre, df_db)

                if id_h:
                    nuevos.append({
                        'fecha_asignacion': fecha,
                        'semana_texto': semana,
                        'id_hermano': id_h,
                        'nombre_hermano': nombre_real,
                        'tipo_asignacion': tipo_dest,
                        'sala': 'Principal'
                    })

    if nuevos:
        df_migrado = pd.DataFrame(nuevos)
        # Modo 'append' seguro: Si existe historial, lo carga y añade
        if os.path.exists(path_out):
            df_existente = pd.read_csv(path_out)
            df_final = pd.concat([df_existente, df_migrado], ignore_index=True)
        else:
            df_final = df_migrado

        df_final.drop_duplicates(subset=['semana_texto', 'tipo_asignacion', 'nombre_hermano'], inplace=True)
        df_final.to_csv(path_out, index=False, encoding='utf-8')

        print(f"\n💾 ¡MIGRACIÓN EXITOSA!")
        print(f"   Se agregaron {len(nuevos)} registros históricos.")
        print(f"   Ahora el sistema conoce tu historial desde Septiembre.")
    else:
        print("⚠️ No se generaron registros. Revisa los nombres de las columnas en tu CSV.")

if __name__ == "__main__":
    ejecutar_migracion()

🚀 INICIANDO MIGRACIÓN v3.0
   ✅ Archivo encontrado: historico_excel_asginaciones.csv

💾 ¡MIGRACIÓN EXITOSA!
   Se agregaron 238 registros históricos.
   Ahora el sistema conoce tu historial desde Septiembre.


# Previo: Inicializar la Memoria (Historial Asginaciones)

In [ ]:
# =============================================================================
#  🛠️ UTILIDAD: INICIALIZAR HISTORIAL (CON AUTO-CONEXIÓN DRIVE)
# =============================================================================

import pandas as pd
import os
from google.colab import drive # Importamos la librería de Drive

def inicializar_historial():
    # --- PASO 1: VERIFICAR CONEXIÓN A DRIVE ---
    if not os.path.exists('/content/drive'):
        print("🔌 Conectando a Google Drive...")
        drive.mount('/content/drive')
    else:
        print("✅ Google Drive ya está conectado.")

    # --- PASO 2: CONFIGURACIÓN DE RUTA ---
    # Verifica que esta ruta sea EXACTAMENTE la de tu carpeta
    RUTA_BASE = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/'
    RUTA_HISTORIAL = os.path.join(RUTA_BASE, 'historial_asignaciones.csv')

    # Verificar si la carpeta existe antes de intentar guardar
    if not os.path.exists(RUTA_BASE):
        print(f"❌ ERROR CRÍTICO: No encuentro la carpeta en Drive.")
        print(f"   Buscaba: {RUTA_BASE}")
        print("   👉 Verifica si el nombre de la carpeta tiene espacios o tildes diferentes.")
        return

    print(f"\n📂 Verificando historial en: {os.path.basename(RUTA_HISTORIAL)}")

    if os.path.exists(RUTA_HISTORIAL):
        print(f"⚠️ El archivo ya existe. No se ha borrado nada por seguridad.")
    else:
        columnas_maestras = [
            'fecha_asignacion', 'semana_texto', 'id_hermano',
            'nombre_hermano', 'tipo_asignacion', 'sala'
        ]

        df_vacio = pd.DataFrame(columns=columnas_maestras)
        df_vacio.to_csv(RUTA_HISTORIAL, index=False, encoding='utf-8')
        print(f"✅ ¡ÉXITO! Archivo de historial creado y listo.")

if __name__ == "__main__":
    inicializar_historial()

🔌 Conectando a Google Drive...
Mounted at /content/drive

📂 Verificando historial en: historial_asignaciones.csv
✅ ¡ÉXITO! Archivo de historial creado y listo.
